Habanero is the Python library for the CrossRef API will need to be installed locally first before running notebook. More info about habanero: https://pypi.org/project/habanero/

In [2]:
#pip install habanero 

In [1]:
from habanero import Crossref
import csv #for handling output
from IPython.display import display
import urllib.request
import urllib.parse
from xml.etree import ElementTree as ET

In [3]:
cr = Crossref()

Should get author info (Last name, First initial) for comparison

In [14]:
oauthorLast = input("Enter last name):")

Enter last name):Sheldon


In [15]:
oauthorFirst = input("Enter first name):")

Enter first name):Nathan


In [16]:
print(oauthorLast, oauthorFirst)

Sheldon Nathan


In [12]:
inputFile = input("Input file name path (ex: ./CV/Sheldon_CV_text_short.txt)):")

Input file name path (ex: ./CV/Sheldon_CV_text_short.txt)):./CV/Sheldon_CV_text_short.txt


In [17]:
print(inputFile)

./CV/Sheldon_CV_text_short.txt


In [24]:
outputFile = "output" + oauthorLast + oauthorFirst + ".txt" 

In [25]:
print(outputFile)

outputSheldonNathan.txt


In [6]:
def cv_parse(l, r):

    cv_dict = dict()
    

    for z in x['message']['items']:
        #z['search-terms']
        
        #cit = z.get('query', None)
        cv_dict['Citation'] = l.rstrip()
        
        authors = z.get('author',None)
        if authors != None:
            authors = ""
            
            for a in z['author']:
                famname = a.get('family',None) 
                if famname == None:
                    famname = "NoName"
                givname = a.get('given',None) 
                if givname == None:
                    givname = "NoName"
                fullname = famname + ", " + givname + "; "
                #print(fullname)
                authors = authors + fullname
            if oauthor in authors:
                cv_dict['Author'] = authors
            else:
                cv_dict['Author'] = "NotAMatch"
        else:
            cv_dict['Author'] = "No Authors Listed"
        
        ppubdate = z.get('published-print',None)
        opubdate = z.get('published-online',None)
        if ppubdate != None:
            cv_dict['Year'] = str(ppubdate['date-parts'][0][0])
            if len(ppubdate['date-parts'][0]) == 1:
                cv_dict['Month-Day'] = None 
            elif len(ppubdate['date-parts'][0]) == 2:
                cv_dict['Month-Day'] = str(ppubdate['date-parts'][0][1])
            elif len(ppubdate['date-parts'][0]) == 3:
                cv_dict['Month-Day'] = str(ppubdate['date-parts'][0][1]) + "-" + str(ppubdate['date-parts'][0][2])
        elif opubdate != None:
            cv_dict['Year'] = str(opubdate['date-parts'][0][0])
            if len(opubdate['date-parts'][0]) == 1:
                cv_dict['Month-Day'] = None 
            elif len(opubdate['date-parts'][0]) == 2:
                cv_dict['Month-Day'] = str(opubdate['date-parts'][0][1])
            elif len(opubdate['date-parts'][0]) == 3:
                cv_dict['Month-Day'] = str(opubdate['date-parts'][0][1]) + "-" + str(opubdate['date-parts'][0][2])
        else:
            cv_dict['Year'] = None

        title = z.get('title', None)
        if title != None:
            cv_dict['Title'] = title[0]
        else:
            cv_dict['Title'] = None

        pubtype = z.get('type',None)
        cv_dict['Type'] = pubtype

        publisher = z.get('publisher',None)
        cv_dict['Publisher'] = publisher

        jname = z.get('container-title',None)
        if jname != None:
            cv_dict['JournalName'] = jname[0]    
        else:
            cv_dict['JournalName'] = None
            

        #cv_dict['Editor'] = editor
        
        editor = z.get('editor',None)
        if editor != None:
            editors = ""
            
            for a in z['editor']:
                famname = a.get('family',None) 
                if famname == None:
                    famname = "NoName"
                givname = a.get('given',None) 
                if givname == None:
                    givname = "NoName"
                fullname = famname + ", " + givname + "; "
                #print(fullname)
                editors = editors + fullname
                cv_dict['Editor'] = editors

        else:
            cv_dict['Editor'] = None

        volume = z.get('volume',None)
        cv_dict['JournalVolume'] = str(volume)

        #Ask about issue?
        issue = z.get('issue',None)
        cv_dict['JournalIssue'] = str(issue)

        page = z.get('page',None)
        if page != None:
            pageSplit = page.split("-")
            cv_dict['Startpage'] = str(pageSplit[0])
            if len(pageSplit) == 1:
                cv_dict['Endpage'] = None
            else:
                cv_dict['Endpage'] = str(pageSplit[1])
        else:
            cv_dict['Startpage'] = None
            cv_dict['Endpage'] = None

        doi = z.get('DOI',None)
        cv_dict['DOI'] = doi

        issn = z.get('issn-type',None)
        if issn != None:
            cv_dict['issn'] = issn[0]['value'] + ", " + issn[0]['type']
        else:
            cv_dict['issn'] = None
        
        if issn != None:
            srdict = dict()
            condlist = list()
            
            #url = 'http://www.sherpa.ac.uk/romeo/api29.php?issn=0001-6918'
            url = 'http://www.sherpa.ac.uk/romeo/api29.php?issn=' + issn[0]['value']
            #print(url)
            
            f = urllib.request.urlopen(url)
            srxml = f.read().decode('utf-8', errors='ignore')
            #print(srxml)
            root = ET.fromstring(srxml)

            for p in root.findall('.//journal'):
                srdict['journalInfo'] = "%s | %s | %s" % (p.find('jtitle').text, p.find('issn').text, p.find('zetocpub').text)

            for c in root.findall('.//publisher/conditions/condition'): # get ALL conditions make this a list then append as needed
                condlist.append(c.text)
            cv_dict['SHERPA-RoMEOc'] = condlist

            for c in root.findall('.//romeocolour'):
                srdict['colour'] = "Colour: %s" % (c.text)

            cv_dict['SHERPA-RoMEOv'] = srdict['journalInfo'] + "; " + srdict['colour']
            
            
            #cv_dict['SHERPA-RoMEO'] = url
    #writer.writerow(cv_dict)   
    return(cv_dict)

In [22]:
with open(inputFile, "r", newline="\n") as input_file, open(outputFile, "w", newline="\n") as output_file:
    #rline = input_file.readline()
    fieldnames = ['Citation', 'Author', 'Year', 'Month-Day', 'Title', 'Type', 'Editor', 'Publisher', 'JournalName', 'JournalVolume', 'JournalIssue', 'Startpage', 'Endpage', 'DOI', 'issn', 'SHERPA-RoMEOv', 'SHERPA-RoMEOc', 'Notes']
    writer = csv.DictWriter(output_file, delimiter = '\t', fieldnames=fieldnames)
    writer.writeheader()
    #writer.writerow(['Citation', 'Author', 'Year', 'Month-Day', 'Title', 'Type', 'Editor', 'Publisher', 'JournalName', 'JournalVolume', 'JournalIssue', 'Startpage', 'Endpage', 'DOI', 'issn', 'SHERPA-RoMEO', 'Notes']) # replace with real field names
    row = 0
    for line in input_file:
        # call function to parse citations
        #print(line)
        x = cr.works(query = line, limit = 1) 
        #print(x['message'])
        cvd = cv_parse(line, x)
        #print(cvd)   
        writer.writerow(cvd)
        row = row + 1
        print(row)

1
2
3
4
5
6
7


In [23]:
import pandas
df = pandas.read_csv('EDSTemplate.txt', sep='\t')
pandas.set_option('display.max_rows', 500)
pandas.set_option('display.max_colwidth', -1)
display(df)

,Citation,Author,Year,Month-Day,Title,Type,Editor,Publisher,JournalName,JournalVolume,JournalIssue,Startpage,Endpage,DOI,issn,SHERPA-RoMEOv,SHERPA-RoMEOc,Notes
0,"Hyland, E.G., Sheldon, N.D., Smith, S.Y., Stromberg, C.A.E., 2019, Late Miocene rise and fall of C4 grasses in the western United States linked to aridification and uplift. GSA Bulletin 131, 224–234. doi:10.1130/B32009.1","Hyland, Ethan G.; Sheldon, Nathan D.; Smith, Selena Y.; Strömberg, Caroline A.E.;",2019,1-1,Late Miocene rise and fall of C4 grasses in the western United States linked to aridification and uplift,journal-article,NaN,Geological Society of America,GSA Bulletin,131,1-2,224.0,234.0,10.1130/b32009.1,"0016-7606, print",Geological Society of America Bulletin | 0016-7606 | Geological Society of America; Colour: white,"[""Publisher's version/PDF cannot be used"", ""On author's personal website or open access repository"", 'Must link to publisher version']",NaN
1,"Hyland, E.G., Huntington, K.W., Sheldon, N.D., Reichgelt, T., 2018, Temperature seasonality in the North American continental interior during the Early Eocene Climatic Optimum: Climate of the Past 14, 1391–1404. doi: 10.5194/cp-13-1391-2018","Hyland, Ethan G.; Huntington, Katharine W.; Sheldon, Nathan D.; Reichgelt, Tammo;",2018,10-4,Temperature seasonality in the North American continental interior during the Early Eocene Climatic Optimum,journal-article,NaN,Copernicus GmbH,Climate of the Past,14,10,1391.0,1404.0,10.5194/cp-14-1391-2018,"1814-9332, electronic",Climate of the Past | 1814-9324 | European Geosciences Union (EGU) / Copernicus Publications; Colour: green,"['Authors retain copyright', 'Creative Commons Attribution License 3.0', 'Eligible UK authors may deposit in <a href=""http://opendepot.org/"" target=""_blank"">OpenDepot</a>', ""Publisher's version/PDF may be used"", 'All titles are open access journals']",NaN
2,"Hyland, E.G., Sheldon, N.D., 2018, Comment on “Paleosol-based paleoclimate reconstruction of the Paleocene-Eocene Thermal Maximum, northern Argentina” by Andrews, E., White, T., del Papa, C. [Palaeogeography, Palaeoclimatology, Palaeoecology 471, 181–195]: Palaeogeography, Palaeoclimatology, Palaeoecology, doi: 10.1016/j.paleo.2017.05.031",NotAMatch,2017,4,"Paleosol-based paleoclimate reconstruction of the Paleocene-Eocene Thermal Maximum, northern Argentina",journal-article,NaN,Elsevier BV,"Palaeogeography, Palaeoclimatology, Palaeoecology",471,None,181.0,195.0,10.1016/j.palaeo.2017.01.042,"0031-0182, print","Palaeogeography, Palaeoclimatology, Palaeoecology | 0031-0182 | Elsevier; Colour: green","['Authors pre-print on any website, including arXiv and RePEC', ""Author's post-print on author's personal website immediately"", 'Author\'s post-print on open access repository after an embargo period of between <num>12</num> <period units=""month"">months</period> and <num>48</num> <period units=""month"">months</period>', 'Permitted deposit due to Funding Body, Institutional and Governmental policy or mandate, may be required to comply with embargo periods of <num>12</num> <period units=""month"">months</period> to <num>48</num> <period units=""month"">months</period>', ""Author's post-print may be used to update arXiv and RepEC"", ""Publisher's version/PDF cannot be used"", 'Must link to publisher version with DOI', ""Author's post-print must be released with a Creative Commons Attribution Non-Commercial No Derivatives License"", 'Publisher last reviewed on 03/06/2015']",NaN
3,"Hyland, E.G., Sheldon, N.D., Cotton, J.M., 2017, Constraining the EECO: a terrestrial interhemispheric comparison: GSA Bulletin 129, 244–252. doi: 10.1130/B31493.1","Hyland, Ethan G.; Sheldon, Nathan D.; Cotton, Jennifer M.;",2017,1,Constraining the early Eocene climatic optimum: A terrestrial interhemispheric comparison,journal-article,NaN,Geological Society of America,Geological Society of America Bulletin,129,1-2,244.0,252.0,10.1130/b31493.1,"0016-7606, print",Geological Society of America Bulletin 

To see individual citation results:

In [8]:
#Optional to see individual citation results

x = cr.works(query = "Sheldon, N.D., Costa, E., Cabrera, L., and Garcés, M., 2012, Continental climatic and weathering response to the Eocene-Oliogocene transition: Journal of Geology 120, 227–236. doi: 10.1086/663984", limit = 1)
print(x['message'])

{'facets': {}, 'total-results': 9697631, 'items': [{'indexed': {'date-parts': [[2019, 2, 17]], 'date-time': '2019-02-17T11:27:59Z', 'timestamp': 1550402879168}, 'reference-count': 46, 'publisher': 'University of Chicago Press', 'issue': '2', 'content-domain': {'domain': [], 'crossmark-restriction': False}, 'short-container-title': ['The Journal of Geology'], 'published-print': {'date-parts': [[2012, 3]]}, 'DOI': '10.1086/663984', 'type': 'journal-article', 'created': {'date-parts': [[2012, 3, 6]], 'date-time': '2012-03-06T16:59:59Z', 'timestamp': 1331053199000}, 'page': '227-236', 'source': 'Crossref', 'is-referenced-by-count': 13, 'title': ['Continental Climatic and Weathering Response to the Eocene-Oligocene Transition'], 'prefix': '10.1086', 'volume': '120', 'author': [{'given': 'Nathan D.', 'family': 'Sheldon', 'sequence': 'first', 'affiliation': []}, {'given': 'Elisenda', 'family': 'Costa', 'sequence': 'additional', 'affiliation': []}, {'given': 'Lluís', 'family': 'Cabrera', 'sequ

SHERPA-RoMEO API, just plug in an ISSN

In [10]:
import urllib.request
import urllib.parse
from xml.etree import ElementTree as ET
url = 'http://www.sherpa.ac.uk/romeo/api29.php?issn=0016-7606'
f = urllib.request.urlopen(url)
srxml = f.read().decode('utf-8')
#print(srxml)
root = ET.fromstring(srxml)

for p in root.findall('.//journal'):
    print("%s | %s | %s" % (p.find('jtitle').text, p.find('issn').text, p.find('zetocpub').text))
    
for c in root.findall('.//publisher/conditions/condition'): # make this a list then append as needed
    print("Conditions: %s" % (c.text))
    
for c in root.findall('.//romeocolour'):
    print("Colour: %s" % (c.text))


Geological Society of America Bulletin | 0016-7606 | Geological Society of America
Conditions: Publisher's version/PDF cannot be used
Conditions: On author's personal website or open access repository
Conditions: Must link to publisher version
Colour: white
